In [ ]:
import pandas as pd

In [ ]:
def merge_all_data_frames(daily_data, firm_characteristics, jkp_factors, linking_table):
    """
    Merges daily returns, firm characteristics, and JKP factor returns into a single unified DataFrame.

    This function:
    - Merges firm-level characteristics (typically quarterly) with a CRSP-Compustat linking table (indexed by 'gvkey')
      that provides PERMNO and PERMCO mappings.
    - Filters the linking table for valid link types and date ranges.
    - Uses `merge_asof` to forward-fill quarterly firm characteristics to match daily frequency.
    - Merges in JKP factor returns (wide format, indexed by month-end 'date' and with one column per factor).
    - Drops temporary or redundant merge columns and sets the final index for analysis.

    Parameters:
    ----------
    daily_data : pd.DataFrame
        DataFrame of daily stock return data. Must contain columns: ['date', 'PERMNO', 'PERMCO'].

    firm_characteristics : pd.DataFrame
        DataFrame of firm-level characteristics at quarterly frequency.
        Must be indexed by ['date', 'gvkey'].

    jkp_factors : pd.DataFrame
        DataFrame of JKP factor returns in wide format, indexed by month-end dates.
        Each column corresponds to a different factor name, and values are the respective returns.

    linking_table : pd.DataFrame
        Compustat-CRSP linking table indexed by 'gvkey'. Must contain columns: ['PERMNO', 'PERMCO', 'LINKTYPE', 'LINKDT', 'LINKENDDT'].

    Returns:
    -------
    pd.DataFrame
        A merged DataFrame indexed by ['date', 'PERMCO', 'PERMNO'], containing:
        - daily returns,
        - forward-filled firm characteristics (from quarterly),
        - monthly JKP factor returns (aligned to the daily data by month-end).
    """
    
    # Reset indexes for merging
    firm_characteristics = firm_characteristics.reset_index()
    linking_table = linking_table.reset_index()

    # Merge firm characteristics with linking table
    comp_linked = pd.merge(firm_characteristics, linking_table, how='left', on='gvkey')

    # Filter for valid link types and valid link dates
    comp_linked = comp_linked[comp_linked['LINKTYPE'].isin(['LU', 'LC'])]
    comp_linked = comp_linked[
        (comp_linked['date'] >= comp_linked['LINKDT']) &
        (comp_linked['date'] <= comp_linked['LINKENDDT'])
    ]

    # Drop PERMCO from comp_linked to avoid duplication
    comp_linked = comp_linked.drop(columns=['PERMCO'], errors='ignore')

    # Rename for clarity and compatibility
    comp_linked.rename(columns={'date': 'quarter_date'}, inplace=True)
    comp_linked['PERMNO'] = comp_linked['PERMNO'].astype('int64')

    # Reset index on daily_data for merge_asof
    daily_data = daily_data.reset_index()

    # Merge using merge_asof to align firm characteristics to most recent available quarter
    merged = pd.merge_asof(
        daily_data.sort_values('date'),
        comp_linked.sort_values('quarter_date'),
        by='PERMNO',
        left_on='date',
        right_on='quarter_date',
        direction='backward'
    )

    # Create month-end column to align with JKP factor data
    merged['month_end'] = merged['date'] + pd.offsets.MonthEnd(0)

    # Reset index on jkp_factors to merge
    jkp_factors = jkp_factors.reset_index()

    # Merge JKP factors by month-end date
    merged = pd.merge(merged, jkp_factors, how='left', left_on='month_end', right_on='date')

    # Drop redundant or intermediary columns
    merged.drop(columns=[
        'quarter_date', 'month_end', 'LINKDT', 'LINKENDDT', 'LINKTYPE', 
        'gvkey', 'date_y'
    ], inplace=True, errors='ignore')

    # Rename date_x back to date for clarity
    merged.rename(columns={'date_x': 'date'}, inplace=True)

    # Set final index as required
    merged.set_index(['date', 'PERMCO', 'PERMNO'], inplace=True)

    return merged